In [1]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
import random
import folium

# --- TTDI bounding box (approx) ---
north, south, east, west = 3.1505, 3.128, 101.641, 101.615
bbox = (west, south, east, north)

# --- Fetch plantable areas (grass, parks, meadows) from OSM ---
tags = {
    'leisure': 'park',
    'landuse': 'grass',
    'natural': 'grass'
}

try:
    plantable_areas = ox.features.features_from_bbox(bbox, tags)
    plantable_areas = plantable_areas.to_crs(epsg=3857)
    plantable_areas = plantable_areas[plantable_areas.geometry.notnull()]
    print(f"Found {len(plantable_areas)} plantable areas")
except Exception as e:
    print(f"Error fetching plantable areas: {e}")
    plantable_areas = gpd.GeoDataFrame(columns=['geometry'], geometry=[])

# --- Generate points inside plantable polygons with tree suggestion ---
plantable_points = []
tree_suggestions = []

for poly in plantable_areas.geometry:
    if poly.is_empty:
        continue
    polygons_to_iterate = [poly] if poly.geom_type == 'Polygon' else list(poly.geoms)
    
    for subpoly in polygons_to_iterate:
        minx, miny, maxx, maxy = subpoly.bounds
        # Determine area in m² for tree suggestion
        area_m2 = subpoly.area
        if area_m2 < 50:
            tree_type = "Small tree (Frangipani / Plumeria)"
        elif area_m2 <= 150:
            tree_type = "Medium tree (Neem / Angsana)"
        else:
            tree_type = "Large tree (Rain Tree / Samanea saman)"
        
        # Generate 5 random points per polygon
        for _ in range(5):
            x = random.uniform(minx, maxx)
            y = random.uniform(miny, maxy)
            pt = Point(x, y)
            if subpoly.contains(pt):
                plantable_points.append(pt)
                tree_suggestions.append(tree_type)

# --- Create GeoDataFrame for plantable points ---
plantable_points_gdf = gpd.GeoDataFrame({
    'geometry': plantable_points,
    'tree': tree_suggestions
}, crs=plantable_areas.crs)

plantable_points_gdf = plantable_points_gdf.to_crs(epsg=4326)
print(f"Generated {len(plantable_points_gdf)} plantable points with tree suggestions")

# --- Center map ---
center_lat = (north + south)/2
center_lon = (east + west)/2
print(f"Map center: {center_lat}, {center_lon}")

# --- Create Folium map ---
m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles='CartoDB positron')

# Add plantable points as blue dots with popup showing tree suggestion
for _, row in plantable_points_gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        color="blue",
        fill=True,
        fill_opacity=0.7,
        popup=row['tree']
    ).add_to(m)

# --- Save map ---
output_path = "/notebooks/ttdi_plantable_trees.html"
m.save(output_path)
print(f"Map saved successfully at {output_path}")

m

Found 23 plantable areas
Generated 65 plantable points with tree suggestions
Map center: 3.13925, 101.628
Map saved successfully at /notebooks/ttdi_plantable_trees.html
